In [0]:
CREATE OR REPLACE TABLE openalex.works.work_topics_frontfill_input
SELECT
  -- count(*) AS total_count
  id as work_id, title, abstract,
  primary_location.source.display_name as journal_name
FROM openalex.works.openalex_works w
LEFT ANTI JOIN openalex.works.work_topics_frontfill lm
  ON w.id = lm.work_id -- changed from topics_key (too slow and not needed in incremental scenarios)
WHERE (length(title) >= 15 OR length(abstract) >= 30)
    --AND length(abstract) > 30
    AND w.id > 6600000000 -- let's focus on XPAC only?
    AND type in ('article', 'book', 'review', 'book-chapter')
    -- AND primary_location.source.display_name IS NOT NULL 
    AND (topics is NULL OR size(topics) = 0);

In [0]:
OPTIMIZE openalex.works.work_topics_frontfill_input ZORDER BY (work_id);

In [0]:
SELECT FORMAT_NUMBER(COUNT(*),0) as num_work_to_process
FROM openalex.works.work_topics_frontfill_input